In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import f_oneway 

## Data Preprocessing

#### 1. Understand the Data:

In [2]:
df1 = pd.read_excel("data1.xlsx")
df2 = pd.read_excel("data2.xlsx")

In [3]:
print(len(df1.columns))
print(df1.columns)
print(len(df2.columns))
print(df2.columns)

26
Index(['PROSPECTID', 'Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL',
       'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M',
       'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl',
       'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL',
       'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL',
       'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL'],
      dtype='object')
62
Index(['PROSPECTID', 'time_since_recent_payment',
       'time_since_first_deliquency', 'time_since_recent_deliquency',
       'num_times_delinquent', 'max_delinquency_level',
       'max_recent_level_of_deliq', 'num_deliq_6mts', 'num_deliq_12mts',
       'num_deliq_6_12mts', 'max_deliq_6mts', 'max_deliq_12mts',
       'num_times_30p_dpd', 'num_times_60p_dpd', 'num_std', 'num_std_6mts',
       'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt',
       'num_dbt_6mts', 'nu

Here's a description of each feature in the df1

1. **PROSPECTID**: Unique identifier for each customer or prospect.
2. **Total_TL**: Total number of trade lines (credit accounts) held by the customer.
3. **Tot_Closed_TL**: Total number of closed trade lines.
4. **Tot_Active_TL**: Total number of active trade lines.
5. **Total_TL_opened_L6M**: Number of trade lines opened in the last 6 months.
6. **Tot_TL_closed_L6M**: Number of trade lines closed in the last 6 months.
7. **pct_tl_open_L6M**: Percentage of trade lines opened in the last 6 months.
8. **pct_tl_closed_L6M**: Percentage of trade lines closed in the last 6 months.
9. **pct_active_tl**: Percentage of active trade lines.
10. **pct_closed_tl**: Percentage of closed trade lines.
11. **Total_TL_opened_L12M**: Number of trade lines opened in the last 12 months.
12. **Tot_TL_closed_L12M**: Number of trade lines closed in the last 12 months.
13. **pct_tl_open_L12M**: Percentage of trade lines opened in the last 12 months.
14. **pct_tl_closed_L12M**: Percentage of trade lines closed in the last 12 months.
15. **Tot_Missed_Pmnt**: Total number of missed payments by the customer.
16. **Auto_TL**: Number of auto loan trade lines.
17. **CC_TL**: Number of credit card trade lines.
18. **Consumer_TL**: Number of consumer loan trade lines.
19. **Gold_TL**: Number of gold loan trade lines.
20. **Home_TL**: Number of home loan trade lines.
21. **PL_TL**: Number of personal loan trade lines.
22. **Secured_TL**: Number of secured loan trade lines.
23. **Unsecured_TL**: Number of unsecured loan trade lines.
24. **Other_TL**: Number of other types of trade lines.
25. **Age_Oldest_TL**: Age of the oldest trade line.
26. **Age_Newest_TL**: Age of the newest trade line.

Here's a description of each feature in the df2:

1. **PROSPECTID**: Unique identifier for each customer or prospect.
2. **time_since_recent_payment**: Time elapsed since the most recent payment.
3. **time_since_first_deliquency**: Time elapsed since the first delinquency.
4. **time_since_recent_deliquency**: Time elapsed since the most recent delinquency.
5. **num_times_delinquent**: Number of times the customer has been delinquent.
6. **max_delinquency_level**: Highest level of delinquency recorded.
7. **max_recent_level_of_deliq**: Highest level of recent delinquency.
8. **num_deliq_6mts**: Number of delinquencies in the last 6 months.
9. **num_deliq_12mts**: Number of delinquencies in the last 12 months.
10. **num_deliq_6_12mts**: Number of delinquencies between 6 and 12 months ago.
11. **max_deliq_6mts**: Maximum level of delinquency in the last 6 months.
12. **max_deliq_12mts**: Maximum level of delinquency in the last 12 months.
13. **num_times_30p_dpd**: Number of times 30+ days past due.
14. **num_times_60p_dpd**: Number of times 60+ days past due.
15. **num_std**: Number of standard loan accounts.
16. **num_std_6mts**: Number of standard loan accounts in the last 6 months.
17. **num_std_12mts**: Number of standard loan accounts in the last 12 months.
18. **num_sub**: Number of subprime loan accounts.
19. **num_sub_6mts**: Number of subprime loan accounts in the last 6 months.
20. **num_sub_12mts**: Number of subprime loan accounts in the last 12 months.
21. **num_dbt**: Number of debt accounts.
22. **num_dbt_6mts**: Number of debt accounts in the last 6 months.
23. **num_dbt_12mts**: Number of debt accounts in the last 12 months.
24. **num_lss**: Number of loss accounts.
25. **num_lss_6mts**: Number of loss accounts in the last 6 months.
26. **num_lss_12mts**: Number of loss accounts in the last 12 months.
27. **recent_level_of_deliq**: Recent level of delinquency.
28. **tot_enq**: Total number of credit inquiries.
29. **CC_enq**: Number of credit card inquiries.
30. **CC_enq_L6m**: Number of credit card inquiries in the last 6 months.
31. **CC_enq_L12m**: Number of credit card inquiries in the last 12 months.
32. **PL_enq**: Number of personal loan inquiries.
33. **PL_enq_L6m**: Number of personal loan inquiries in the last 6 months.
34. **PL_enq_L12m**: Number of personal loan inquiries in the last 12 months.
35. **time_since_recent_enq**: Time elapsed since the most recent inquiry.
36. **enq_L12m**: Number of inquiries in the last 12 months.
37. **enq_L6m**: Number of inquiries in the last 6 months.
38. **enq_L3m**: Number of inquiries in the last 3 months.
39. **MARITALSTATUS**: Marital status of the customer.
40. **EDUCATION**: Educational level of the customer.
41. **AGE**: Age of the customer.
42. **GENDER**: Gender of the customer.
43. **NETMONTHLYINCOME**: Net monthly income of the customer.
44. **Time_With_Curr_Empr**: Time spent with the current employer.
45. **pct_of_active_TLs_ever**: Percentage of active trade lines ever held.
46. **pct_opened_TLs_L6m_of_L12m**: Percentage of trade lines opened in the last 6 months relative to the last 12 months.
47. **pct_currentBal_all_TL**: Percentage of current balance across all trade lines.
48. **CC_utilization**: Utilization rate of credit card trade lines.
49. **CC_Flag**: Indicator if the customer has credit card trade lines.
50. **PL_utilization**: Utilization rate of personal loan trade lines.
51. **PL_Flag**: Indicator if the customer has personal loan trade lines.
52. **pct_PL_enq_L6m_of_L12m**: Percentage of personal loan inquiries in the last 6 months relative to the last 12 months.
53. **pct_CC_enq_L6m_of_L12m**: Percentage of credit card inquiries in the last 6 months relative to the last 12 months.
54. **pct_PL_enq_L6m_of_ever**: Percentage of personal loan inquiries in the last 6 months relative to all personal loan inquiries ever.
55. **pct_CC_enq_L6m_of_ever**: Percentage of credit card inquiries in the last 6 months relative to all credit card inquiries ever.
56. **max_unsec_exposure_inPct**: Maximum unsecured exposure as a percentage of total credit.
57. **HL_Flag**: Indicator if the customer has home loan trade lines.
58. **GL_Flag**: Indicator if the customer has gold loan trade lines.
59. **last_prod_enq2**: Time elapsed since the last product inquiry.
60. **first_prod_enq2**: Time elapsed since the first product inquiry.
61. **Credit_Score**: Credit score of the customer.
62. **Approved_Flag**: Indicator if the customer's credit application was approved.

#### 2. Data Cleaning:

In [4]:
# There are some values like -99999 in both df1 and df2 handling them

In [5]:
def columns_with_99999(df):
    columns_to_remove = []
    filter_columns = []
    for i in df.columns:
        if df.loc[df[i] == -99999].shape[0] > 10000: # removing the entire col
            columns_to_remove.append(i)
        elif df.loc[df[i] == -99999].shape[0] > 0: # removing only the rows
            filter_columns.append(i)
    
    return columns_to_remove,filter_columns

In [6]:
columns_to_remove_in_df1 , filter_columns_df1 = columns_with_99999(df1)
columns_to_remove_in_df2 , filter_columns_df2 = columns_with_99999(df2)

print(columns_to_remove_in_df1)
print(filter_columns_df1)
print(columns_to_remove_in_df1)
print(filter_columns_df2)

[]
['Age_Oldest_TL', 'Age_Newest_TL']
[]
['time_since_recent_payment', 'tot_enq', 'CC_enq', 'CC_enq_L6m', 'CC_enq_L12m', 'PL_enq', 'PL_enq_L6m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L12m', 'enq_L6m', 'enq_L3m', 'pct_currentBal_all_TL']


In [7]:
# before
print(df1.shape)
print(df2.shape)

(51336, 26)
(51336, 62)


In [8]:
df1.drop(columns_to_remove_in_df1 , axis= 1 , inplace=True)
df2.drop(columns_to_remove_in_df2 , axis= 1 , inplace=True)

In [9]:
for i in filter_columns_df1:
    df1 = df1.loc[df1[i] != -99999]

for i in filter_columns_df2:
    df2 = df2.loc[df2[i] != -99999]


In [10]:
# after
print(df1.shape)
print(df2.shape)

(51296, 26)
(42066, 54)


In [11]:
# checking any null value
print(df1.isna().sum().sum())
print(df2.isna().sum().sum())

0
0


Now joining df1 and df2 into df

In [12]:
# checking the common columns
for i in df1.columns:
    for j in df2.columns:
        if i == j:
            print(i)

PROSPECTID


In [13]:
# performing inner join on df1 and df2 to avoid null values
df = pd.merge(df1,df2,on='PROSPECTID',how='inner')

In [14]:
df.shape

(42064, 79)

#### 3. Data Transformation:

##### 3.1 Data filtering

In [15]:
#identifying numerical and categorical columns

numerical_columns = []
categorical_columns = []

for i in df.columns:
    if df[i].dtype == 'object':
        categorical_columns.append(i)
    else:
        numerical_columns.append(i)

In [16]:
categorical_columns

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

In [17]:
# removing Approved_Flag as this is target 
categorical_columns.remove('Approved_Flag')

In [18]:
# seeing all value counts
for i in categorical_columns:
    print(df[i].value_counts())
    print("---------------------------------")

MARITALSTATUS
Married    30886
Single     11178
Name: count, dtype: int64
---------------------------------
EDUCATION
GRADUATE          14140
12TH              11703
SSC                7241
UNDER GRADUATE     4572
OTHERS             2291
POST-GRADUATE      1898
PROFESSIONAL        219
Name: count, dtype: int64
---------------------------------
GENDER
M    37345
F     4719
Name: count, dtype: int64
---------------------------------
last_prod_enq2
ConsumerLoan    16480
others          13653
PL               7553
CC               2195
AL               1353
HL                830
Name: count, dtype: int64
---------------------------------
first_prod_enq2
others          20640
ConsumerLoan    11075
PL               4431
AL               2641
CC               1988
HL               1289
Name: count, dtype: int64
---------------------------------


Checking association between two categorical variables (CHI-SQUARE TEST)

since all the categorical columns p-value <= 0.05 so we will select all columns

In [19]:
# Chi-square test
from scipy.stats import chi2_contingency
for i in categorical_columns:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, '---', pval)

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


Variance Inflation Factor (VIF) is a measure used to detect multicollinearity in regression analysis.

In [20]:
# VIF for numerical columns
numeric_columns = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

Multicollinearity vs Correlation

Multicollinearity = Predictability of each features by other features

Correlation is specific to linear realtionships between columns
in convex function (y = X^2), corelation give misleading values

__Multicolinearity__

Variance Inflation Factor

Used to identify multicollinearity among IVs

Takes R-squared value for each IV and eliminate if crosses a threshold
1/(1-R^2)

VIF ranges from 1 from infinity

1 : No multicollinearity

1 to 5 : Low multicollinearity

5 to 10 :Moderate multicollinearity

VIF above 10 : High multicollinearity

In [21]:
# VIF sequentially check
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = df[numeric_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0



for i in range (0,total_columns):
    
    vif_value = variance_inflation_factor(vif_data, column_index)
    print (column_index,'---',vif_value)
    
    
    if vif_value <= 6:
        columns_to_be_kept.append( numeric_columns[i] )
        column_index = column_index+1
    
    else:
        vif_data = vif_data.drop([ numeric_columns[i] ] , axis=1)


c:\Python312\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


c:\Python312\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000336
0 --- 6.520647877790928
0 --- 5.149501618212625
1 --- 2.611111040579735


c:\Python312\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.8328007921530785
3 --- 6.099653381646739
3 --- 5.581352009642762
4 --- 1.985584353098778


c:\Python312\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf
5 --- 4.809538302819343
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.3843464059655854
7 --- 3.064658415523423
8 --- 2.898639771299253
9 --- 4.377876915347324
10 --- 2.207853583695844
11 --- 4.916914200506864
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721438


c:\Python312\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf
15 --- 7.380634506427232
15 --- 1.4210050015175733
16 --- 8.083255010190316
16 --- 1.6241227524040112
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.825857047132431
18 --- 1.5080839450032664
19 --- 2.172088834824577
20 --- 2.62339755352723
21 --- 2.2959970812106176
22 --- 7.360578319196439
22 --- 2.1602387773102554
23 --- 2.8686288267891458
24 --- 6.458218003637277
24 --- 2.8474118865638265
25 --- 4.753198156284083
26 --- 16.22735475594825
26 --- 6.424377256363877
26 --- 8.887080381808687
26 --- 2.3804746142952653
27 --- 8.609513476514548
27 --- 13.06755093547673
27 --- 3.500040056654654
28 --- 1.9087955874813773
29 --- 17.006562234161628
29 --- 10.730485153719197
29 --- 2.3538497522950275
30 --- 22.104855915136433
30 --- 2.7971639638512906
31 --- 3.4241712032176985
32 --- 10.175021454450935
32 --- 6.408710354561301
32 --- 1.001151196262561
33 --- 3.069197305397274
34 --- 2.8091261600643715
35 --- 20.249538381980678
35 --- 15.864576541593774
35 --- 1.833164974053

In [22]:
len(columns_to_be_kept)

39

In [23]:
# check Anova for columns_to_be_kept 

from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])  
    b = list(df['Approved_Flag'])  
    
    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)

In [24]:
len(columns_to_be_kept_numerical)

37

In [25]:
# listing all the final features
features = columns_to_be_kept_numerical + categorical_columns
df = df[features + ['Approved_Flag']] # adding the target variable

##### 3.2 Encode categorical features (e.g., one-hot encoding, label encoding).

In [26]:
print(df['MARITALSTATUS'].unique())
print(df['EDUCATION'].unique())
print(df['GENDER'].unique())
print(df['last_prod_enq2'].unique())
print(df['first_prod_enq2'].unique())

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
['M' 'F']
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


So we can only label encode education column as other columns doesnot make any sense. for eg for 'MARITALSTATUS' there are 2 unique values = ['Married' 'Single'] and making married = 1 and Single = 0 does not make any sense.

In [27]:
# Ordinal feature -- EDUCATION
# SSC            : 1
# 12TH           : 2
# GRADUATE       : 3
# UNDER GRADUATE : 3
# POST-GRADUATE  : 4
# OTHERS         : 1
# PROFESSIONAL   : 3


# Others has to be verified by the business end user 

**Label encoding**

In [28]:
# manual label encoding
df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3

In [29]:
df['EDUCATION'] = df['EDUCATION'].astype(int)

**One hot encoding**

In [30]:
# One hot encoding
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])

In [31]:
df_encoded.to_csv('cleaned_data.csv')